1. Criar a State Machine no Step Functions
Primeiro, você precisa criar uma State Machine no Step Functions que define a ordem de execução das Lambdas. Aqui está um exemplo de como você pode fazer isso:

Arquivo: step_functions/criar_step_functions.py

In [ ]:
import boto3
import json


def criar_state_machine(nome_state_machine, role_arn, lambda_upload_arn, lambda_textract_arn, lambda_mover_arn):
    client = boto3.client('stepfunctions')

    # Definição da State Machine (fluxo de trabalho)
    state_machine_definition = {
        "Comment": "Orquestração do processamento de notas fiscais",
        "StartAt": "LambdaUpload",
        "States": {
            "LambdaUpload": {
                "Type": "Task",
                "Resource": lambda_upload_arn,
                "Next": "LambdaTextract"
            },
            "LambdaTextract": {
                "Type": "Task",
                "Resource": lambda_textract_arn,
                "Next": "LambdaMover"
            },
            "LambdaMover": {
                "Type": "Task",
                "Resource": lambda_mover_arn,
                "End": True
            }
        }
    }

    try:
        response = client.create_state_machine(
            name=nome_state_machine,
            definition=json.dumps(state_machine_definition),
            roleArn=role_arn
        )
        print(f"State Machine '{nome_state_machine}' criada com sucesso!")
        return response['stateMachineArn']
    except Exception as e:
        print(f"Erro ao criar State Machine: {e}")
        return None

2. Integrar o API Gateway com o Step Functions
Agora, você precisa modificar o script do API Gateway para integrá-lo diretamente com o Step Functions. O API Gateway invocará a State Machine, e o Step Functions cuidará da orquestração das Lambdas.

Arquivo: api/api_gateway.py

In [ ]:
import boto3
import json
from botocore.exceptions import NoCredentialsError, ClientError


def criar_api_gateway(nome_api, state_machine_arn, role_arn):
    client = boto3.client('apigateway')

    try:
        # Cria o API Gateway
        response = client.create_rest_api(
            name=nome_api,
            description='API Gateway para processamento de notas fiscais',
            version='1.0'
        )
        api_id = response['id']

        # Obtém o ID do recurso raiz
        resources = client.get_resources(restApiId=api_id)
        root_resource_id = resources['items'][0]['id']

        # Cria um recurso para o endpoint
        resource_response = client.create_resource(
            restApiId=api_id,
            parentId=root_resource_id,
            pathPart='processar'
        )
        resource_id = resource_response['id']

        # Cria o método POST para o recurso
        client.put_method(
            restApiId=api_id,
            resourceId=resource_id,
            httpMethod='POST',
            authorizationType='NONE'
        )

        # Integra o método POST com o Step Functions
        client.put_integration(
            restApiId=api_id,
            resourceId=resource_id,
            httpMethod='POST',
            type='AWS',
            integrationHttpMethod='POST',
            uri=f'arn:aws:apigateway:us-east-1:states:action/StartExecution',
            credentials=role_arn,  # Role ARN com permissão para invocar Step Functions
            requestTemplates={
                'application/json': json.dumps({
                    "stateMachineArn": state_machine_arn,
                    "input": "$util.escapeJavaScript($input.json('$'))"
                })
            }
        )

        # Implanta o API Gateway
        client.create_deployment(
            restApiId=api_id,
            stageName='prod'
        )

        print(f"API Gateway '{nome_api}' criado com sucesso!")
        return api_id

    except NoCredentialsError:
        print("Credenciais não encontradas.")
        return None
    except ClientError as e:
        print(f"Erro ao criar API Gateway: {e}")
        return None

3. Atualizar o main.py
Agora, você precisa atualizar o main.py para criar a State Machine e integrá-la ao API Gateway:

Arquivo: main.py

In [ ]:
from infra.infra import criar_infraestrutura
from lambdas.criar_lambdas import criar_lambda_upload, criar_lambda_textract, criar_lambda_mover
from step_functions.criar_step_functions import criar_state_machine
from api.api_gateway import criar_api_gateway


def main():
    # Criar a infraestrutura (buckets, roles, políticas)
    print("Criando infraestrutura...")
    role_arn, bucket_lambda_code_name, bucket_imagens_name = criar_infraestrutura()

    # Criar as funções Lambda
    print("Criando funções Lambda...")
    lambda_upload_arn = criar_lambda_upload(role_arn, bucket_lambda_code_name)
    lambda_textract_arn = criar_lambda_textract(
        role_arn, bucket_lambda_code_name)
    lambda_mover_arn = criar_lambda_mover(role_arn, bucket_lambda_code_name)

    # Criar a State Machine no Step Functions
    print("Criando State Machine...")
    state_machine_arn = criar_state_machine(
        nome_state_machine="ProcessamentoNotasFiscais",
        role_arn=role_arn,
        lambda_upload_arn=lambda_upload_arn,
        lambda_textract_arn=lambda_textract_arn,
        lambda_mover_arn=lambda_mover_arn
    )

    if state_machine_arn:
        print(f"State Machine criada com ARN: {state_machine_arn}")
    else:
        print("Falha ao criar State Machine.")
        return

    # Criar o API Gateway
    print("Criando API Gateway...")
    api_id = criar_api_gateway('NotasFiscaisAPI', state_machine_arn, role_arn)

    if api_id:
        print(f"API Gateway criado com ID: {api_id}")
    else:
        print("Falha ao criar API Gateway.")

    print("Processo concluído com sucesso!")


if __name__ == "__main__":
    main()

4. Permissões e Roles
Certifique-se de que a Role usada pelo API Gateway tenha permissões para invocar o Step Functions. Adicione a seguinte política à Role:

In [ ]:
{
    "Effect": "Allow",
    "Action": "states:StartExecution",
    "Resource": "*"
}

5. Fluxo de Execução
O cliente faz uma requisição POST para o endpoint do API Gateway.

O API Gateway invoca a State Machine no Step Functions.

A State Machine orquestra a execução das Lambdas na ordem definida:

Primeiro, a Lambda de upload.

Depois, a Lambda do Textract.

Por fim, a Lambda de mover arquivos.